<a href="https://colab.research.google.com/github/marcellomazzochi/dio-lab-open-source/blob/main/pyspark_taxi_yellow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=80eb6008c8247f494d9c5efc078cc464f8e0217503959fd90ea1b30d096053e1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!pip install wget
!pip install findspark

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=220010473c105bf8f3010a1107c5f17147b6a668ec974bca18f2c3df6773fb8a
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
from pyspark.sql.window import Window
import re
import os

In [ ]:
import findspark

findspark.init()

In [ ]:
spark = SparkSession.builder.appName("TaxiAnalysis").getOrCreate()


In [ ]:
!mkdir -p pasta_parquet

!wget -O yellow_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!wget -O yellow_tripdata_2022-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
!wget -O yellow_tripdata_2022-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet
!wget -O yellow_tripdata_2022-04.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet
!wget -O yellow_tripdata_2022-05.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet
!wget -O yellow_tripdata_2022-06.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet
!wget -O yellow_tripdata_2022-07.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-07.parquet
!wget -O yellow_tripdata_2022-08.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-08.parquet
!wget -O yellow_tripdata_2022-09.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-09.parquet
!wget -O yellow_tripdata_2022-10.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-10.parquet
!wget -O yellow_tripdata_2022-11.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-11.parquet
!wget -O yellow_tripdata_2022-12.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-12.parquet
!wget -O /content/taxi_zone_lookup.csv https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

!mv yellow_tripdata_2022-*.parquet pasta_parquet/

--2024-07-07 00:20:36--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.163.157.133, 3.163.157.7, 3.163.157.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.163.157.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  17.8MB/s    in 2.0s    

2024-07-07 00:20:39 (17.8 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]

--2024-07-07 00:20:39--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.163.157.133, 3.163.157.7, 3.163.157.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.163.157.133|:443... connected.
HT

In [ ]:
!ls /content/pasta_parquet  # Deverá mostrar 'pasta_parquet' e talvez outros arquivos


yellow_tripdata_2022-01.parquet  yellow_tripdata_2022-05.parquet  yellow_tripdata_2022-09.parquet
yellow_tripdata_2022-02.parquet  yellow_tripdata_2022-06.parquet  yellow_tripdata_2022-10.parquet
yellow_tripdata_2022-03.parquet  yellow_tripdata_2022-07.parquet  yellow_tripdata_2022-11.parquet
yellow_tripdata_2022-04.parquet  yellow_tripdata_2022-08.parquet  yellow_tripdata_2022-12.parquet


In [ ]:
yellow_tripdata_df = spark.read.parquet("/content/pasta_parquet/yellow_tripdata_2022-*.parquet")

# Carregar os dados de lookup de zona de táxi
taxi_zone_lookup_df = spark.read.csv("/content/taxi_zone_lookup.csv", header=True, inferSchema=True)

# Filtrar viagens de 2022
filtered_data = yellow_tripdata_df.filter(year("tpep_pickup_datetime") == 2022)

# Realizar join entre yellow_tripdata e taxi_zone_lookup com base em DOLocationID e LocationID
joined_df = filtered_data.join(taxi_zone_lookup_df, filtered_data.DOLocationID == taxi_zone_lookup_df.LocationID)

# Selecionar colunas necessárias, extrair o mês e contar as viagens por VendorID, mês, Borough e DOLocationID
aggregated_df = joined_df.select(
    "VendorID",
    month("tpep_pickup_datetime").alias("month"),
    "Borough",
    "DOLocationID"
).groupBy("VendorID", "month", "Borough").agg(
    count("DOLocationID").alias("count")
)

# Usar window function para encontrar o DOLocationID mais frequente por VendorID e mês
window_spec = Window.partitionBy("VendorID", "month").orderBy(col("count").desc())
result_df = aggregated_df.withColumn("rank", rank().over(window_spec)).filter(col("rank") == 1)

# Selecionar as colunas finais e mostrar os resultados
final_df = result_df.select("VendorID", "Borough", "month", "count").orderBy("VendorID", "month")
final_df.show()


+--------+---------+-----+-------+
|VendorID|  Borough|month|  count|
+--------+---------+-----+-------+
|       1|Manhattan|    1| 667217|
|       1|Manhattan|    2| 804657|
|       1|Manhattan|    3| 955646|
|       1|Manhattan|    4| 932534|
|       1|Manhattan|    5| 935181|
|       1|Manhattan|    6| 923917|
|       1|Manhattan|    7| 805781|
|       1|Manhattan|    8| 777611|
|       1|Manhattan|    9| 798752|
|       1|Manhattan|   10| 935121|
|       1|Manhattan|   11| 662457|
|       1|Manhattan|   12| 835756|
|       2|Manhattan|    1|1544045|
|       2|Manhattan|    2|1877736|
|       2|Manhattan|    3|2278202|
|       2|Manhattan|    4|2258539|
|       2|Manhattan|    5|2210646|
|       2|Manhattan|    6|2201578|
|       2|Manhattan|    7|1979882|
|       2|Manhattan|    8|1977520|
+--------+---------+-----+-------+
only showing top 20 rows



In [ ]:
import pyspark.sql.functions as F


In [ ]:
yellow_tripdata_df_2022 = yellow_tripdata_df.filter(year(col("tpep_pickup_datetime")) == 2022)

# Realizar a união com a tabela taxi_zone_lookup com base no DOLocationID e LocationID
joined_df = yellow_tripdata_df_2022.join(taxi_zone_lookup_df, yellow_tripdata_df_2022.DOLocationID == taxi_zone_lookup_df.LocationID, "inner")

# Adicionar colunas de ano e mês
joined_df = joined_df.withColumn("year", year(col("tpep_pickup_datetime")))
joined_df = joined_df.withColumn("month", month(col("tpep_pickup_datetime")))

# Agrupar por VendorID, Borough e month, e contar o número de ocorrências
grouped_df = joined_df.groupBy("VendorID", "Borough", "month").agg(count("*").alias("count"))

# Utilizar janela para obter o DOLocationID mais frequente por mês para cada VendorID
window_spec = Window.partitionBy("VendorID", "month").orderBy(col("count").desc())

# Adicionar a coluna rank para determinar o DOLocationID mais frequente
ranked_df = grouped_df.withColumn("rank", F.rank().over(window_spec))

# Filtrar apenas os registros com rank 1 (mais frequente)
most_frequent_df = ranked_df.filter(col("rank") == 1).select("VendorID", "Borough", "count", "month")

# Mostrar o resultado
most_frequent_df.show()


+--------+---------+-------+-----+
|VendorID|  Borough|  count|month|
+--------+---------+-------+-----+
|       1|Manhattan| 667217|    1|
|       1|Manhattan| 804657|    2|
|       1|Manhattan| 955646|    3|
|       1|Manhattan| 932534|    4|
|       1|Manhattan| 935181|    5|
|       1|Manhattan| 923917|    6|
|       1|Manhattan| 805781|    7|
|       1|Manhattan| 777611|    8|
|       1|Manhattan| 798752|    9|
|       1|Manhattan| 935121|   10|
|       1|Manhattan| 662457|   11|
|       1|Manhattan| 835756|   12|
|       2|Manhattan|1544045|    1|
|       2|Manhattan|1877736|    2|
|       2|Manhattan|2278202|    3|
|       2|Manhattan|2258539|    4|
|       2|Manhattan|2210646|    5|
|       2|Manhattan|2201578|    6|
|       2|Manhattan|1979882|    7|
|       2|Manhattan|1977520|    8|
+--------+---------+-------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import year, month, col

# Carregar os dados de viagem
yellow_tripdata_df = spark.read.parquet("/content/pasta_parquet/yellow_tripdata_2022-*.parquet")

# Carregar os dados de lookup de zona de táxi
taxi_zone_lookup_df = spark.read.csv("/content/taxi_zone_lookup.csv", header=True, inferSchema=True)

# Filtrar viagens de 2022
filtered_trips = yellow_tripdata_df.filter(year("tpep_pickup_datetime") == 2022)

# Join das tabelas por DOLocationID e LocationID
joined_df = filtered_trips.join(
    taxi_zone_lookup_df,
    filtered_trips.DOLocationID == taxi_zone_lookup_df.LocationID,
    "inner"
)

# Selecionar colunas necessárias e extrair o mês do campo datetime
selected_df = joined_df.select(
    "VendorID",
    "Borough",
    month("tpep_pickup_datetime").alias("month"),
    "DOLocationID"
)

# Agrupar por VendorID, Borough, e mês, e contar as viagens
grouped_df = selected_df.groupBy("VendorID", "Borough", "month").count()

# Registrar o DataFrame como uma tabela temporária para usar spark.sql
grouped_df.createOrReplaceTempView("trip_summary")

# Consultar os top 5 destinos por taxista por mês usando Spark SQL
result_df = spark.sql("""
SELECT VendorID, Borough, month, count
FROM (
    SELECT VendorID, Borough, month, count,
           ROW_NUMBER() OVER (PARTITION BY VendorID, month ORDER BY count DESC) as rank
    FROM trip_summary
) ranked
WHERE rank <= 5
""")

result_df.show()


In [ ]:
yellow_tripdata_df = spark.read.parquet("/content/pasta_parquet/yellow_tripdata_2022-*.parquet")

# Carregar os dados de lookup de zona de táxi
taxi_zone_lookup_df = spark.read.csv("/content/taxi_zone_lookup.csv", header=True, inferSchema=True)

# Filtrar viagens de 2022
yellow_tripdata_df_2022 = yellow_tripdata_df.filter(year(col("tpep_pickup_datetime")) == 2022)

# Realizar a união com a tabela taxi_zone_lookup com base no DOLocationID e LocationID
joined_df = yellow_tripdata_df_2022.join(
    taxi_zone_lookup_df,
    yellow_tripdata_df_2022.DOLocationID == taxi_zone_lookup_df.LocationID,
    "inner"
)

# Adicionar colunas de ano e mês
joined_df = joined_df.withColumn("year", year(col("tpep_pickup_datetime")))
joined_df = joined_df.withColumn("month", month(col("tpep_pickup_datetime")))

# Agrupar por VendorID, Borough e month, e contar o número de ocorrências
grouped_df = joined_df.groupBy("VendorID", "Borough", "month").agg(count("*").alias("count"))

# Utilizar janela para obter o rank por contagem de viagens
window_spec = Window.partitionBy("VendorID", "month").orderBy(col("count").desc())

# Adicionar a coluna rank para determinar os top destinos mais frequentes
ranked_df = grouped_df.withColumn("rank", F.rank().over(window_spec))

# Filtrar apenas os registros com rank 1 (mais frequente) e VendorID igual a 1
most_frequent_df = ranked_df.filter((col("rank") == 1) & (col("VendorID") == 1))

# Selecionar colunas relevantes
final_df = most_frequent_df.select("VendorID", "Borough", "count", "month")

# Mostrar o resultado
final_df.show()


In [ ]:
yellow_tripdata_df_2022 = yellow_tripdata_df.filter(year(col("tpep_pickup_datetime")) == 2022)
joined_df = yellow_tripdata_df_2022.join(taxi_zone_lookup_df, yellow_tripdata_df_2022.DOLocationID == taxi_zone_lookup_df.LocationID)


In [ ]:
yellow_tripdata_df_2022 = yellow_tripdata_df.filter(year(col("tpep_pickup_datetime")) == 2022)
joined_df = yellow_tripdata_df_2022.join(taxi_zone_lookup_df, yellow_tripdata_df_2022.DOLocationID == taxi_zone_lookup_df.LocationID)
aggregated_df = joined_df.select(
    "VendorID",
    month("tpep_pickup_datetime").alias("month"),
    "Borough",
    "DOLocationID"
).groupBy("VendorID", "month", "Borough").agg(
    count("DOLocationID").alias("count")
)

# Usar window function para encontrar o DOLocationID mais frequente por VendorID e mês
window_spec = Window.partitionBy("VendorID", "month").orderBy(col("count").desc())
result_df = aggregated_df.withColumn("rank", rank().over(window_spec)).filter(col("rank") == 1)

# Selecionar as colunas finais e mostrar os resultados
final_df = result_df.select("VendorID", "Borough", "month", "count").orderBy("VendorID", "month")
final_df.show()


In [ ]:
joined_df = joined_df.withColumn("year", year(col("tpep_pickup_datetime")))
joined_df = joined_df.withColumn("month", month(col("tpep_pickup_datetime")))

# Agrupar por VendorID, Borough e month, e contar o número de ocorrências
grouped_df = joined_df.groupBy("VendorID", "Borough", "month").agg(count("*").alias("count"))

# Utilizar janela para obter o DOLocationID mais frequente por mês para cada VendorID
window_spec = Window.partitionBy("VendorID", "month").orderBy(col("count").desc())

# Adicionar a coluna rank para determinar o DOLocationID mais frequente
ranked_df = grouped_df.withColumn("rank", F.rank().over(window_spec))

# Filtrar apenas os registros com rank 1 (mais frequente)
most_frequent_df = ranked_df.filter(col("rank") == 1).select("VendorID", "Borough", "count", "month")

# Mostrar o resultado
most_frequent_df.show()


In [ ]:
window_spec = Window.partitionBy("month").orderBy(col("trip_count").desc())
yellow_tripdata_df_2022 = yellow_tripdata_df_2022.withColumn("month", month("tpep_pickup_datetime"))

# Contagem de viagens por mês e DOLocationID, aplicação do ranking
top_destinations = joined_df.groupBy("month", "DOLocationID") \
                            .count() \
                            .withColumnRenamed("count", "trip_count") \
                            .withColumn("rank", rank().over(window_spec)) \
                            .filter(col("rank") <= 5)  # Considerando os top-3 destinos

# Mostrar os resultados
top_destinations.show()


+-----+------------+----------+----+
|month|DOLocationID|trip_count|rank|
+-----+------------+----------+----+
|    1|         236|    123467|   1|
|    1|         237|    106353|   2|
|    1|         141|     78339|   3|
|    1|         239|     75052|   4|
|    1|         161|     74707|   5|
|    2|         236|    139050|   1|
|    2|         237|    124630|   2|
|    2|         161|     99171|   3|
|    2|         170|     88876|   4|
|    2|         239|     88384|   5|
|    3|         236|    157834|   1|
|    3|         237|    147557|   2|
|    3|         161|    134942|   3|
|    3|         170|    111112|   4|
|    3|         230|    106047|   5|
|    4|         236|    154968|   1|
|    4|         237|    148961|   2|
|    4|         161|    131122|   3|
|    4|         230|    109244|   4|
|    4|         170|    105305|   5|
+-----+------------+----------+----+
only showing top 20 rows



In [ ]:
yellow_tripdata_df_2022 = yellow_tripdata_df.withColumn("month", month("tpep_pickup_datetime"))

# Definindo a especificação da janela para ranking de destinos por mês
window_spec = Window.partitionBy("month").orderBy(col("trip_count").desc())

# Contagem de viagens por mês e DOLocationID, aplicação do ranking
top_destinations = yellow_tripdata_df_2022.groupBy("month", "DOLocationID") \
                                          .count() \
                                          .withColumnRenamed("count", "trip_count") \
                                          .withColumn("rank", rank().over(window_spec))

# Join com a tabela de lookup de zona para incluir o nome do bairro
top_destinations_with_borough = top_destinations.join(taxi_zone_lookup_df,
                                                      top_destinations.DOLocationID == taxi_zone_lookup_df.LocationID,
                                                      "left_outer") \
                                                .select("month", "DOLocationID", "Borough", "trip_count", "rank")

# Filtrando para mostrar apenas os top 5 destinos por mês
top_5_destinations = top_destinations_with_borough.filter(col("rank") <= 5)

# Mostrar os resultados
top_5_destinations.show()


+-----+------------+---------+----------+----+
|month|DOLocationID|  Borough|trip_count|rank|
+-----+------------+---------+----------+----+
|    1|         236|Manhattan|    123469|   1|
|    1|         237|Manhattan|    106354|   2|
|    1|         141|Manhattan|     78339|   3|
|    1|         239|Manhattan|     75052|   4|
|    1|         161|Manhattan|     74708|   5|
|    2|         236|Manhattan|    139050|   1|
|    2|         237|Manhattan|    124630|   2|
|    2|         161|Manhattan|     99171|   3|
|    2|         170|Manhattan|     88876|   4|
|    2|         239|Manhattan|     88384|   5|
|    3|         236|Manhattan|    157834|   1|
|    3|         237|Manhattan|    147557|   2|
|    3|         161|Manhattan|    134942|   3|
|    3|         170|Manhattan|    111112|   4|
|    3|         230|Manhattan|    106047|   5|
|    4|         236|Manhattan|    154968|   1|
|    4|         237|Manhattan|    148961|   2|
|    4|         161|Manhattan|    131122|   3|
|    4|      

In [ ]:
joined_df = yellow_tripdata_df.join(taxi_zone_lookup_df, yellow_tripdata_df.DOLocationID == taxi_zone_lookup_df.LocationID)

# Agrupar por mês, DOLocationID e Borough, e contar os VendorIDs únicos
monthly_destination_stats = joined_df.groupBy("month", "DOLocationID", "Borough") \
                                     .agg(count(col("VendorID")).alias("taxist_count"))

# Definir a janela de partição por mês e ordenar por taxist_count decrescente
window_spec = Window.partitionBy("month").orderBy(desc("taxist_count"))

# Aplicar a função rank e filtrar os top 5 destinos por mês
top_destinations = monthly_destination_stats.withColumn("rank", rank().over(window_spec)) \
                                            .filter(col("rank") <= 5)

# Mostrar os resultados
top_destinations.select("month", "DOLocationID", "Borough", "taxist_count").show()


+-----+------------+---------+------------+
|month|DOLocationID|  Borough|taxist_count|
+-----+------------+---------+------------+
|    1|         236|Manhattan|      123449|
|    1|         237|Manhattan|      106348|
|    1|         141|Manhattan|       78326|
|    1|         239|Manhattan|       75037|
|    1|         161|Manhattan|       74698|
|    2|         236|Manhattan|      139057|
|    2|         237|Manhattan|      124627|
|    2|         161|Manhattan|       99171|
|    2|         170|Manhattan|       88869|
|    2|         239|Manhattan|       88392|
|    3|         236|Manhattan|      157814|
|    3|         237|Manhattan|      147542|
|    3|         161|Manhattan|      134937|
|    3|         170|Manhattan|      111093|
|    3|         230|Manhattan|      106035|
|    4|         236|Manhattan|      154956|
|    4|         237|Manhattan|      148957|
|    4|         161|Manhattan|      131115|
|    4|         230|Manhattan|      109231|
|    4|         170|Manhattan|  

In [ ]:
!mkdir -p pasta_shapefile

!wget -O /content/pasta_shapefile/taxi_zones.zip https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip
!unzip -o /content/pasta_shapefile/taxi_zones.zip -d /content/pasta_shapefile/


In [ ]:
import geopandas as gpd
gdf = gpd.read_file("/content/pasta_shapefile/taxi_zones.shp")

# Calcular adjacências
gdf['adjacent'] = gdf.apply(lambda row: gdf[gdf.geometry.touches(row.geometry)]['LocationID'].tolist(), axis=1)

# Exibir os resultados
print(gdf[['LocationID', 'adjacent']])


     LocationID                  adjacent
0             1                        []
1             2                     [132]
2             3       [51, 184, 242, 254]
3             4            [79, 148, 224]
4             5                        []
..          ...                       ...
258         259                     [254]
259         260                     [157]
260         261            [88, 209, 231]
261         262       [75, 140, 141, 263]
262         263  [75, 140, 141, 236, 262]

[263 rows x 2 columns]


In [ ]:
most_visited_destinations = yellow_tripdata_df \
    .groupBy("month", col("DOLocationID")) \
    .agg(count("*").alias("trip_count")) \
    .withColumn("rank", rank().over(window_spec)) \
    .filter(col("rank") <= 5) \
    .drop("rank")

# Juntar com os dados de lookup para obter o Borough
most_visited_destinations_with_borough = most_visited_destinations.join(taxi_zone_lookup_df,
                                                                       most_visited_destinations.DOLocationID == taxi_zone_lookup_df.LocationID,
                                                                       "inner") \
    .select("month", "DOLocationID", "Borough", "trip_count")

# Carregar os dados de geolocalização e calcular adjacências
gdf = gpd.read_file("/content/pasta_shapefile/taxi_zones.shp")
gdf['adjacent'] = gdf.apply(lambda row: gdf[gdf.geometry.touches(row.geometry)]['LocationID'].tolist(), axis=1)

# Converter o GeoDataFrame para um DataFrame do Pandas
adjacency_df = gdf[['LocationID', 'adjacent']].copy()

# Converter o DataFrame do Pandas para um DataFrame do Spark
adjacency_spark_df = spark.createDataFrame(adjacency_df)

# Juntar os dados de adjacência ao resultado final
final_result = most_visited_destinations_with_borough.join(adjacency_spark_df,
                                                           most_visited_destinations_with_borough.DOLocationID == adjacency_spark_df.LocationID,
                                                           "inner") \
    .select("month", "DOLocationID", "Borough", "trip_count", "adjacent")

# Mostrar os resultados
final_result.orderBy("month", "trip_count", ascending=[True, False]).show(truncate=False)


+-----+------------+---------+----------+------------------------------+
|month|DOLocationID|Borough  |trip_count|adjacent                      |
+-----+------------+---------+----------+------------------------------+
|1    |236         |Manhattan|123449    |[141, 237, 263]               |
|1    |237         |Manhattan|106348    |[43, 141, 162, 163, 229, 236] |
|1    |141         |Manhattan|78326     |[162, 236, 237, 262, 263]     |
|1    |239         |Manhattan|75037     |[142, 238]                    |
|1    |161         |Manhattan|74698     |[100, 162, 163, 164, 170]     |
|2    |236         |Manhattan|139057    |[141, 237, 263]               |
|2    |237         |Manhattan|124627    |[43, 141, 162, 163, 229, 236] |
|2    |161         |Manhattan|99171     |[100, 162, 163, 164, 170]     |
|2    |170         |Manhattan|88869     |[107, 137, 161, 162, 164, 234]|
|2    |239         |Manhattan|88392     |[142, 238]                    |
|3    |236         |Manhattan|157814    |[141, 237,